In [149]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import pandas as pd
import itertools
import importlib
import mlintro_min as mli

In [150]:
%matplotlib inline

##### To anticipate section 2 on nilearn: please download the data below if not done already

**Note:** please run these commands from a different notebook / terminal to avoid having your Jupyter kernel busy

In [ ]:
from nilearn import datasets
# Replace "datasets/age_netmats" with whatever path you would like
data = datasets.fetch_development_fmri(data_dir='datasets/age_netmats')
atlases = datasets.fetch_atlas_basc_multiscale_2015(version='sym', data_dir='datasets/age_netmats')

# Simple regression (OLS) model

Range of cannon angle psi to investigate

In [ ]:
psi_min=20
psi_max=60

Read pre-generated source of data

In [ ]:
ref_df = pd.read_csv('datasets/offline/refdata_100K.csv')
# Get a very small subset of inter-spread data for plotting
ref_df_light = mli.get_ref_light(ref_df, psi_min=psi_min, psi_max=psi_max)

Generate a single dataset of 50 observations

In [ ]:
np.random.seed(42)
ds = mli.get_datasets(ref_df, n_datasets=1, sample_size=50, psi_min=psi_min, psi_max=psi_max)[0]

##### Explore dataset

We are only interested in experimental measures of angle and range, `exp_angle` and `exp_range` respectively

In [ ]:
ds

Plot dataset

In [ ]:
plt.figure(figsize=(4, 4))
sns.scatterplot("exp_angle", "exp_range", data=ds, color='blue')
plt.ylim([50, 80]);

## Part 1: model selection with train / test split

We will compare three models of different complexity (different number of parameters):
   * Standard linear model (2 parameters: $\beta_0$, $\beta_\psi$)
   * Linear model with the feature squared (3 parameters: $\beta_0$, $\beta_\psi$, $\beta_{\psi^2}$)
   * Linear model with up to $5^{th}$ power of feature (6 parameters: $\beta_0$, $\beta_\psi$, $\beta_{\psi^2}$, $\beta_{\psi^3}$, $\beta_{\psi^4}$, $\beta_{\psi^5}$)

In [ ]:
from sklearn.model_selection import train_test_split

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

Let's divide our dataset into training and test set !

In [ ]:
X = ds[['exp_angle']]
y = ds['exp_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
print("Number of observations: {} for training and {} for testing".format(len(X_train), len(X_test)))

In [ ]:
X_train

### Linear (OLS) model with single feature

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

##### Let's train the model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

Create new estimator object (each model is an object in `sklearn`)

In [ ]:
lm = LinearRegression()

In [ ]:
lm.

In [ ]:
lm.fit(X_train, y_train)

In [ ]:
lm.

In [ ]:
print('beta_psi: {}, beta_0: {}'.format(lm.coef_, lm.intercept_))

##### Predict, and compute performance scores

In [ ]:
lm.predict([[30], [45], [60]])

The training score can provide an idea of best possible performance

In [ ]:
# We score the predictions made on the same data we trained on
y_pred = lm.predict(X_train)

R2_train = r2_score(y_train, y_pred)
MSE_train = mean_squared_error(y_train, y_pred)
print('lm training performance is R2: {:0.2f} and MSE: {:0.2f}'.format(R2_train, MSE_train))

`R2` can be obtained directly from the model / estimator object as it is the default for `LinearRegression`

In [ ]:
lm.score(X_train, y_train)

Let's save all our models scores to compare models later

In [ ]:
train_test_results = []
train_test_results.append({'model': 'lm', 'stage': 'train', 'scorer': 'r2', 'val': R2_train})
train_test_results.append({'model': 'lm', 'stage': 'train', 'scorer': 'MSE', 'val': -MSE_train})

##### Let's test on unseen data and get testing score

See possible scores: https://scikit-learn.org/stable/modules/model_evaluation.html

The testing score can provide an idea of performance for unseen data

In [ ]:
# We score the predictions made on unseen data
y_pred = lm.predict(X_test)

R2_test = r2_score(y_test, y_pred)
MSE_test = mean_squared_error(y_test, y_pred)
print('lm testing performance is R2: {:0.2f} and MSE: {:0.2f}'.format(R2_test, MSE_test))

Again let's save our scores for comparison later

In [ ]:
train_test_results.append({'model': 'lm', 'stage': 'test', 'scorer': 'r2', 'val': R2_test})
train_test_results.append({'model': 'lm', 'stage': 'test', 'scorer': 'MSE', 'val': -MSE_test})

### Polynomial degree 2 model (polynomial use feature powers as additional features)

##### Train and get training score of degree 2 polynomial

The polynomial terms are created and then added to a standard `LinearRegression`

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

In [ ]:
poly_transformer = PolynomialFeatures(degree=2)
lm_deg2 = LinearRegression()

A polynomial model is fitted by first transforming the features and then training a `LinearRegression` on the transformed features

In [ ]:
X_train_deg2 = poly_transformer.fit_transform(X_train)

In [ ]:
X_train

In [ ]:
X_train_deg2

In [ ]:
lm_deg2.fit(X_train_deg2, y_train)

In [ ]:
lm_deg2.coef_

Let's compute training score  
**Warning**: `LinearRegression` needs to be applied to the *transformed* (polynomial) features

In [ ]:
y_pred = lm_deg2.predict(X_train_deg2)

R2_train = r2_score(y_train, y_pred)
MSE_train = mean_squared_error(y_train, y_pred)
print('poly deg2 training performance is R2: {:0.2f} and MSE: {:0.2f}'.format(R2_train, MSE_train))

In [ ]:
train_test_results.append({'model': 'lm_deg2', 'stage': 'train', 'scorer': 'r2', 'val': R2_train})
train_test_results.append({'model': 'lm_deg2', 'stage': 'train', 'scorer': 'MSE', 'val': -MSE_train})

##### Let's test on unseen data and get testing score

Again, should not forget to transform the test data to get polynomial features

In [ ]:
X_test_deg2 = poly_transformer.fit_transform(X_test) 

y_pred = lm_deg2.predict(X_test_deg2)

R2_test = r2_score(y_test, y_pred)
MSE_test = mean_squared_error(y_test, y_pred)
print('poly deg2 testing performance is R2: {:0.2f} and MSE: {:0.2f}'.format(R2_test, MSE_test))

In [ ]:
train_test_results.append({'model': 'lm_deg2', 'stage': 'test', 'scorer': 'r2', 'val': R2_test})
train_test_results.append({'model': 'lm_deg2', 'stage': 'test', 'scorer': 'MSE', 'val': -MSE_test})

### Polynomial degree 5 model, introducing pipeline object

It is common to use preprocessing steps such as features transformation. To avoid repeating these processing steps (e.g. when testing the model on new data) the `Pipeline` object is very useful. It builds a workflow which can be called with a single command. 

##### Train and get training score of degree 5 polynomial with *pipeline*

In [ ]:
from sklearn.pipeline import Pipeline

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

The creation of the polynomial terms are embed in the `Pipeline` object which represents our new estimator (i.e. the one to use with `fit`, `predict`, etc.)

In [ ]:
lm_deg5 = Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                    ('lm', LinearRegression())])
lm_deg5.fit(X_train, y_train)

In [ ]:
lm_deg5['lm'].coef_

In [ ]:
# Let's compute training score
y_pred = lm_deg5.predict(X_train)

R2_train = r2_score(y_train, y_pred)
MSE_train = mean_squared_error(y_train, y_pred)
print('poly deg5 training performance is R2: {:0.2f} and MSE: {:0.2f}'.format(R2_train, MSE_train))

In [ ]:
train_test_results.append({'model': 'lm_deg5', 'stage': 'train', 'scorer': 'r2', 'val': R2_train})
train_test_results.append({'model': 'lm_deg5', 'stage': 'train', 'scorer': 'MSE', 'val': -MSE_train})

##### Get score on test data

No more transformation to do manually as it is embed in the pipeline!

In [ ]:
y_pred = lm_deg5.predict(X_test)
R2_test = r2_score(y_test, y_pred)
MSE_test = mean_squared_error(y_test, y_pred)
print('poly deg5 testing performance is R2: {:0.2f} and MSE: {:0.2f}'.format(R2_test, MSE_test))

In [ ]:
train_test_results.append({'model': 'lm_deg5', 'stage': 'test', 'scorer': 'r2', 'val': R2_test})
train_test_results.append({'model': 'lm_deg5', 'stage': 'test', 'scorer': 'MSE', 'val': -MSE_test})

In [ ]:
train_test_results_df = pd.DataFrame(train_test_results)
r2_results = train_test_results_df.loc[train_test_results_df['scorer'] == 'r2']
MSE_results = train_test_results_df.loc[train_test_results_df['scorer'] == 'MSE']
with sns.plotting_context("notebook", font_scale=1.2):
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=train_test_results_df, 
                    kind="bar", sharey=False)

For many other regression models implemented in sklearn, cf https://scikit-learn.org/stable/supervised_learning.html#supervised-learning 

## Part 2: Model selection with cross-validation

https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
KFold?

Let's see what K fold is actually doing on a 10-observation dataset

In [ ]:
X_example = pd.DataFrame({'obs': 
                          np.random.randint(0, 1000, 10)})
X_example

In [ ]:
kf = KFold(n_splits=5)
for (ix_train, ix_test) in kf.split(X_example):
    print("train ix:", ix_train, "testix:", ix_test)

In [ ]:
kfs = KFold(n_splits=5, shuffle=True, random_state=42)
for (ix_train, ix_test) in kfs.split(X_example):
    print("train ix:", ix_train, "test ix:", ix_test)

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print("Size X: {}, with X train: {} and X test: {}".format(len(X), len(X_train), len(X_test)))

In [ ]:
ml_models = {'lm': LinearRegression(),
             'lm_deg2': Pipeline([('poly_transformer', PolynomialFeatures(degree=2)),
                                  ('lm', LinearRegression())]),
             'lm_deg5': Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                                  ('lm', LinearRegression())])}

In [ ]:
#from sklearn.model_selection import StratifiedKFold

In [ ]:
kf_results = []

kfs = KFold(n_splits=10, shuffle=True, random_state=42)
for i_f, (ix_train, ix_test) in enumerate(kfs.split(X_train)):
    # Loop over models
    for mod_name in ml_models.keys():
        # Fit the model on the training folds
        ml_models[mod_name].fit(X_train.iloc[ix_train], y_train.iloc[ix_train])
        # Test on both the training and testing folds to check for over-/under-fitting
        y_pred_train = ml_models[mod_name].predict(X_train.iloc[ix_train])
        y_pred_test = ml_models[mod_name].predict(X_train.iloc[ix_test])
        # R2
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'train', 'scorer': 'r2', 
                           'val': r2_score(y_train.iloc[ix_train], y_pred_train)})
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'test', 'scorer': 'r2', 
                           'val': r2_score(y_train.iloc[ix_test], y_pred_test)})
        # MSE
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'train', 'scorer': 'MSE', 
                           'val': -mean_squared_error(y_train.iloc[ix_train], y_pred_train)})
        kf_results.append({'model': mod_name, 'fold': i_f, 'stage': 'test', 'scorer': 'MSE', 
                           'val': -mean_squared_error(y_train.iloc[ix_test], y_pred_test)})
kf_results_df = pd.DataFrame(kf_results)

In [ ]:
for mod_name in ml_models.keys():
    kf_df = kf_results_df.loc[kf_results_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=kf_df, kind="bar", 
                        sharey=False, height=3, aspect=1.5)
        g.axes[0,0].set_ylim(-0.2,1)
        #g.axes[0,1].set_ylim(0, 45)
        g.fig.suptitle(mod_name, y=1.05)

In [ ]:
with sns.plotting_context("notebook", font_scale=1.2):
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=kf_results_df, kind="box",
                    sharey=False, height=3, aspect=1.5)
    g.axes[0, 0].set_ylim(-0.2, 1)
    g.axes[0, 1].set_ylim(-50, 0)
    g.fig.suptitle("Score accross fold across models", y=1.05)
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=kf_results_df, kind="box", 
                    sharey=False, height=3, aspect=1.5)
    g.axes[0, 0].set_ylim(0.45, 1)
    g.axes[0, 1].set_ylim(-18, 0)
    g.fig.suptitle("Close up on polynomial models", y=1.05)

#### `sklearn` helps to automate common operations 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score?

In [ ]:
from sklearn.metrics import fbeta_score, make_scorer
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
ml_models = {'lm': LinearRegression(),
             'lm_deg2': Pipeline([('poly_transformer', PolynomialFeatures(degree=2)),
                                  ('lm', LinearRegression())]),
             'lm_deg5': Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                                  ('lm', LinearRegression())])}
# Get cv train AND test scores
cv_test_scores = {}
for mod_name in ml_models.keys():
    cv_test_scores[mod_name] = cross_val_score(ml_models[mod_name], X_train, y_train, cv=kfs,
                                               scoring=mse_scorer, n_jobs=-1)
cv_test_scores_df = pd.DataFrame(cv_test_scores)

In [ ]:
cv_test_scores_df.boxplot()
plt.title('Negative MSE for the three models tested (larger is better)');

##### Want even more with even less code?

In [ ]:
from sklearn.model_selection import cross_validate

`cross_validate` returns train *and* test scores on *several* metrics

In [ ]:
cross_validate?

In [ ]:
ml_models = {'lm': LinearRegression(),
             'lm_deg2': Pipeline([('poly_transformer', PolynomialFeatures(degree=2)),
                                  ('lm', LinearRegression())]),
             'lm_deg5': Pipeline([('poly_transformer', PolynomialFeatures(degree=5)),
                                  ('lm', LinearRegression())])}
# Get cv train AND test scores
cv_scores = {}
for mod_name in ml_models.keys():
    cv_scores[mod_name] = cross_validate(ml_models[mod_name], X_train, y_train, cv=kfs, 
                                         scoring=['r2', 'neg_mean_squared_error'], n_jobs=-1,
                                         return_train_score=True)

In [151]:
def crossval_to_df(cv_dict):
    crossval_results = []
    for model in cv_dict.keys():
        for scorer in cv_dict[model].keys():
            if scorer.startswith('train_'):
                score = scorer.replace('train_', '')
                for i_val, val in enumerate(cv_dict[model][scorer]):
                    crossval_results.append({'model': model, 'fold': i_val, 'stage': 'train', 
                                             'scorer': score, 'val': val})
            elif scorer.startswith('test_'):
                score = scorer.replace('test_', '')
                for i_val, val in enumerate(cv_dict[model][scorer]):
                    crossval_results.append({'model': model, 'fold': i_val, 'stage': 'test', 
                                             'scorer': score, 'val': val})
    return pd.DataFrame(crossval_results)

In [ ]:
crossval_df = crossval_to_df(cv_scores)
for mod_name in crossval_df['model'].unique():
    kf_df = crossval_df.loc[crossval_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=crossval_df,
                        kind="bar", sharey=False, height=3, aspect=1.5, ci=None)
        g.axes[0,0].set_ylim(-0.2,1)
        g.axes[0,1].set_ylim(-45, 0)
        g.fig.suptitle(mod_name, y=1.05)

# Penalized regression and feature selection

We will use our new skills to compare models for predicting age from brain functional connectivity matrices. We will use data ready to be directly analyzed with machine learning. We will see later how to generate these data with `nilearn`. We will compare in this example:
* Feature selection + classic linear regression
* Penalized regression

**Feature selection** is useful for:
* Helping prevent overfitting
* Obtaining parsimonious models
* Speeding up calculations

**Note:** This section is using the same MRI data example as in the notebook Jake Vogel presented at the 2020 Brainhack School: https://github.com/neurodatascience/course-materials-2020/blob/master/lectures/14-may/03-intro-to-machine-learning/ML_Regression_Tutorial.ipynb  

The nilearn part here is very similar while the machine learning part is quite different (different subset of data, different models). 

##### Get the data

In [ ]:
from nilearn import datasets

In [ ]:
datasets.fetch_development_fmri?

Better starting the download now in a different Jupyter notebook / terminal.

In [ ]:
data = datasets.fetch_development_fmri(data_dir='datasets/age_netmats')

## Part 1: individual model evaluation

### ML data preparation

Create the feature matrix `X` and labels `y`

In [ ]:
features_file = 'datasets/offline/MAIN_BASC064_subsamp_features.npz'
X_all = np.load(features_file)['a']

In [ ]:
no_download = True
if no_download:
    pheno_file = 'datasets/offline/pheno.csv'
    pheno_df=pd.read_csv(pheno_file)
else:
    pheno_df = pd.DataFrame(data.phenotypic)

Let's have a look at `X_all` (we should always look at our data when we can)

In [ ]:
plt.figure(figsize=(16,16))
plt.imshow(X_all, aspect='auto')
plt.colorbar()
plt.title('feature matrix')
plt.xlabel('features')
plt.ylabel('subjects')

In [ ]:
plt.figure(figsize=(16,16))
g = sns.heatmap(X_all, vmin=-1, vmax=1, cmap='viridis')
g.set(xlabel='Features', ylabel='Subjects', title='Feature matrix');

In [ ]:
pheno_df.iloc[36:63]

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(np.std(X_all, axis=1), label = 'std')
median_s_std = np.median(np.std(X_all, axis=1))
std_s_std = np.std(np.std(X_all, axis=1))
min_s_std = np.min(np.std(X_all, axis=1))
max_s_std = np.max(np.std(X_all, axis=1))
plt.axhline(median_s_std, label='median std', color='g')
plt.vlines(np.where(pheno_df["Age"]<5)[0], ymin=min_s_std, ymax=max_s_std, color='r', alpha=0.4,
           label='child less than 5 years old')
plt.title('Random QA plot')
plt.legend();

In [ ]:
pheno_df['AgeGroup'].value_counts()

In [ ]:
# Plotting hist without kde
ax = sns.distplot(pheno_df[['Age']], kde=False)
ax2 = ax.twinx()
sns.distplot(pheno_df[['Age']], ax=ax2, kde=True, hist=False)
ax2.set_yticks([]);

In [ ]:
print("Number of subjects less than 13: {}".format(pheno_df['Age'].lt(13).sum()))
print("Number of subjects more than 13: {}".format(pheno_df['Age'].ge(13).sum()))
print("Total number of subjects: {}".format(len(pheno_df)))

#### Get data in shape

Let's look at models predicting age only for children until 12 (could do same exercise with also removing children less than 5)

In [ ]:
lt13_boolmask = (pheno_df['Age'] < 15)
y = pheno_df[['Age']].loc[lt13_boolmask]
X = X_all[lt13_boolmask, :]
age_class = pheno_df[['AgeGroup']].loc[lt13_boolmask]
print("Number of subjects is {} and number of features is {}".format(*X.shape))

#### Some serious overfitting ahead if we don't do anything!

Let's split our dataset to have round numbers (easier to conceptualize CV we just studied).Options to `train_test_split` are:
* 100/22 train/test subjects 
* shuffle, very important as data has some order!
* stratification to make sure under-represented 8-12 group balanced in all folds
* random see to reproduce the results

In [ ]:
train_test_split?

In [ ]:
tts_ix = np.arange(len(X))
X_train, X_test, y_train, y_test, age_class_train, age_class_test, X_ix, y_ix = train_test_split(
    X, y, age_class, tts_ix, test_size=22, shuffle = True, stratify=age_class, random_state = 42)
print("Number of subjects: training={}, testing={}".format(len(y_train), len(y_test)))

Let's make sure the stratified splitting worked out

In [ ]:
sns.distplot(y_train, label='train')
sns.distplot(y_test, label='test')
plt.legend();

### Classic lineal (OLS) model

Just for the sake of confirming what we know: what should we get for training and testing performance if we fit a standard linear regression model with 100 datapoints and 2000+ features? Let's find out.

In [ ]:
lm = LinearRegression()

#### Evaluate model with (stratified!) cross-validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

**Option 1**: manually (as the stratification can only be done on label data `y`, not on `age_class_train`)

In [ ]:
skf_results = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for i_f, (ix_train, ix_test) in enumerate(skf.split(X_train, age_class_train)):
        lm.fit(X_train[ix_train], y_train.iloc[ix_train])
        # Test on both the testing fold and training folds to check for over-/under-fitting
        y_pred_train = lm.predict(X_train[ix_train])
        y_pred_test = lm.predict(X_train[ix_test])
        # R2
        skf_results.append({'model': 'lm', 'fold': i_f, 'stage': 'train', 'scorer': 'r2', 
                            'val': r2_score(y_train.iloc[ix_train], y_pred_train)})
        skf_results.append({'model': 'lm', 'fold': i_f, 'stage': 'test', 'scorer': 'r2', 
                           'val': r2_score(y_train.iloc[ix_test], y_pred_test)})
        # MSE
        skf_results.append({'model': 'lm', 'fold': i_f, 'stage': 'train', 'scorer': 'MSE', 
                           'val': -mean_squared_error(y_train.iloc[ix_train], y_pred_train)})
        skf_results.append({'model': 'lm', 'fold': i_f, 'stage': 'test', 'scorer': 'MSE', 
                           'val': -mean_squared_error(y_train.iloc[ix_test], y_pred_test)})
skf_results_df = pd.DataFrame(skf_results)

**Option 2**: create custom cv based on `age_class_train`

In [ ]:
class AgeStratifiedKFold(StratifiedKFold):
    def __init__(self, n_splits=5, *, shuffle=False, random_state=None, age_grp=age_class_train):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
        self.age_grp = age_grp
    
    def split(self, X, y, groups=None):
        local_age_grp = self.age_grp.loc[y.index.to_list()]
        return super().split(X, local_age_grp, groups)

In [ ]:
age_skf = AgeStratifiedKFold(n_splits=10, shuffle=True, random_state=42, age_grp=age_class_train)

In [ ]:
cv_scores = {}
cv_scores['lm'] = cross_validate(lm, X_train, y_train, cv=age_skf, 
                                 scoring=['r2', 'neg_mean_squared_error'], n_jobs=-1,
                                 return_train_score=True)
cv_scores_df = pd.DataFrame(cv_scores)
crossval_df = crossval_to_df(cv_scores)

#### PLot the results

In [ ]:
skf_df = skf_results_df.loc[skf_results_df['model'] == 'lm']
with sns.plotting_context("notebook", font_scale=1.2):
    # Barploat
    g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=skf_df, kind="bar", 
                    sharey=False, height=3, aspect=1.5)
    g.axes[0,0].set_ylim(-0.2,1)
    g.fig.suptitle('lm', y=1.05)
    # Boxplot
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=skf_results_df, kind="box",
                    sharey=False, height=3, aspect=1.5)
    g.axes[0, 0].set_ylim(-0.2, 1)

(Below are identical plots from the custom cv solution )

In [ ]:
for mod_name in crossval_df['model'].unique():
    kf_df = crossval_df.loc[crossval_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        # Bar plot
        g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=crossval_df,
                        kind="bar", sharey=False, height=3, aspect=1.5, ci=None,
                        hue_order =["train", "test"])
        g.axes[0,0].set_ylim(-0.2,1)
        g.fig.suptitle(mod_name, y=1.05)
        # Box plot
        g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=crossval_df, 
                        kind="box", sharey=False, height=3, aspect=1.5, hue_order =["train", "test"])
        g.axes[0, 0].set_ylim(-0.2, 1)

### Adding feature selection to OLS model

https://scikit-learn.org/stable/modules/feature_selection.html

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

Let's look into two different types of feature selection and create the associated pipelines 

In [ ]:
# F-score (correlation with target turned into F score then pvalue)
fs_freg = SelectKBest(f_regression, k=10)
lm_fs_freg = Pipeline([('fs_freg', fs_freg), 
                       ('lm', LinearRegression())])
# Mutual information score
fs_mireg = SelectKBest(mutual_info_regression, k=10)
lm_fs_mireg = Pipeline([('fs_mireg', fs_mireg), 
                        ('lm', LinearRegression())])

In [ ]:
SelectKBest?

#### Let's do a simple application of one of our pipeline with the default hyper-parameter K for K-best (k=10)

In [ ]:
age_skf = AgeStratifiedKFold(n_splits=10, shuffle=True, random_state=42, age_grp=age_class_train)
cv_scores = {}
for ml_model in [lm_fs_freg, lm_fs_mireg]:
    cv_scores[ml_model] = cross_validate(ml_model, X_train, y_train, cv=age_skf,
                                         scoring=['r2', 'neg_mean_squared_error'],
                                         return_train_score=True, n_jobs=-1)
cv_scores_df = pd.DataFrame(cv_scores)
crossval_df = crossval_to_df(cv_scores)

In [ ]:
# For visualization, rename model names appropriately
def rename_model(model_name):
    if str(model_name).startswith('Pipeline'):
        return '___'.join([s[0] for s in model_name.steps])
    else:
        return model_name
crossval_df['model'] = crossval_df['model'].apply(rename_model)

###### Plotting

In [ ]:
for mod_name in crossval_df['model'].unique():
    kf_df = crossval_df.loc[crossval_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        # Bar plot
        g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=kf_df,
                        kind="bar", sharey=False, height=3, aspect=1.5, ci=None,
                        hue_order =["train", "test"])
        g.axes[0,0].set_ylim(-0.2,1)
        g.fig.suptitle(mod_name, y=1.05)
        # Box plot
        g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=kf_df, 
                        kind="box", sharey=False, height=3, aspect=1.5,
                        hue_order =["train", "test"])
        g.axes[0,0].set_ylim(-0.2,1)

#### Now looking at range of hyper-parameter

In [ ]:
from sklearn.model_selection import validation_curve

In [ ]:
validation_curve?

**Warning:** the cell below can take some time to run. Could be useful to reduce the range of the `k` hyperparameter (e.g. [5, 15, 25, 50, 75, 100]) if you don't have a lot of CPU resources.

In [ ]:
k_range = [1, 5, 10, 15, 20, 25, 50, 75, 100, 150, 200, 250, 500]
# Feature selection with f-score
f_train_r2, f_test_r2 = validation_curve(lm_fs_freg, X_train, y_train, param_name="fs_freg__k", 
                                         param_range=k_range, cv=age_skf, scoring='r2', n_jobs=-1)
f_train_mse, f_test_mse = validation_curve(lm_fs_freg, X_train, y_train, param_name="fs_freg__k", 
                                           param_range=k_range, cv=age_skf, 
                                           scoring='neg_mean_squared_error', n_jobs=-1)
# Feature selection with MI-score
mi_train_re, mi_test_r2 = validation_curve(lm_fs_mireg, X_train, y_train, param_name="fs_mireg__k", 
                                           param_range=k_range, cv=age_skf, scoring='r2', n_jobs=-1)
mi_train_mse, mi_test_mse = validation_curve(lm_fs_mireg, X_train, y_train, param_name="fs_mireg__k", 
                                             param_range=k_range, cv=age_skf, 
                                             scoring='neg_mean_squared_error', n_jobs=-1)

In [152]:
def valcurve_to_df(train_scores, test_scores, param_range, model, scorer):
    valcurve_results = []
    n_folds = train_scores.shape[1]
    for i_param, param in enumerate(param_range):
        valcurve_results.append(pd.DataFrame(
            {'score': train_scores[i_param], 'stage': ['train']*n_folds,
             'fold': np.arange(n_folds), 'param': np.array([param]*n_folds),
             'model': [model]*n_folds, 'scorer': [scorer]*n_folds}))
        valcurve_results.append(pd.DataFrame(
            {'score': test_scores[i_param], 'stage': ['test']*n_folds,
             'fold': np.arange(n_folds), 'param': np.array([param]*n_folds),
             'model': [model]*n_folds, 'scorer': [scorer]*n_folds}))
    return pd.concat(valcurve_results)

In [ ]:
valcurve_df = pd.concat([valcurve_to_df(f_train_r2, f_test_r2, k_range, 'lm_fs_freg', 'r2'),
                         valcurve_to_df(f_train_mse, f_test_mse, k_range, 'lm_fs_freg', 'mse'),
                         valcurve_to_df(mi_train_re, mi_test_r2, k_range, 'lm_fs_mireg', 'r2'),
                         valcurve_to_df(mi_train_mse, mi_test_mse, k_range, 'lm_fs_mireg', 'mse')])

#### Hyper-parameter evaluation results

In [ ]:
for mod_name in valcurve_df['model'].unique():
    kf_df = valcurve_df.loc[valcurve_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        g = sns.catplot(x='param', y='score', hue='stage', col='scorer', data=kf_df, kind='point',
                        sharey=False, hue_order =["train", "test"], height=3, aspect=1.5)
        #plt.xticks(range(10))
        g.set_xticklabels(k_range, rotation=90)
        g.axes[0,0].set_ylim(-0.2, 1.1)
        g.axes[0,1].set_ylim(-45, 1)
        g.fig.suptitle(mod_name, y=1.05)

In [ ]:
from sklearn.model_selection import cross_val_predict

Look at relationship between predicted and real with `cross_val_predict`

In [ ]:
%%capture output --no-stdout --no-display
# F-score + OLS pipeline
fs_freg = SelectKBest(f_regression, k=10)
lm_fs_freg = Pipeline([('fs_freg', fs_freg), 
                       ('lm', LinearRegression())])
y_pred = cross_val_predict(lm_fs_freg, X_train, y_train, cv=age_skf)

In [ ]:
%%capture output --no-stdout --no-display

# Scores
r2 = r2_score(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
print("Scores from CV are R2: {}, MSE: {}".format(r2, mse))

sns.regplot(y_pred, y_train)
plt.xlabel('Predicted Age with f-score feature elimination + OLS')
plt.ylabel('True age')

Now transforming the predicted variable to account for the possible power law  
https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html

In [ ]:
from sklearn.compose import TransformedTargetRegressor

In [ ]:
# F-score + OLS pipeline + log transformer
fs_freg = SelectKBest(f_regression, k=10)
lm_fs_freg = Pipeline([('fs_freg', fs_freg), 
                       ('lm', LinearRegression())])
ttr = TransformedTargetRegressor(regressor=lm_fs_freg, func=np.log, inverse_func=np.exp)

y_pred = cross_val_predict(ttr, X_train, y_train, cv=age_skf)

In [ ]:
# Scores
r2 = r2_score(np.log(y_train), np.log(y_pred))
mse = mean_squared_error(np.log(y_train), np.log(y_pred))
print("Scores from CV are R2: {}, MSE: {}".format(r2, mse))

sns.regplot(np.log(y_pred), np.log(y_train))
plt.xlabel('Predicted Age with f-score feature elimination + OLS')
plt.ylabel('True age')

### Using penalized regression

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(random_state=42, max_iter=10000)
alpha_range = np.logspace(-4, -0.5, 30)
alpha_range

Let's investigate the change in performance score (R2, MSE) as we vary the penalty parameters alpha

**Warning:** the cell below can take some time to run. Could be useful to reduce the range of the `alpha` hyperparameter (e.g. `alpha_range = np.logspace(-2, -0.5, 15)`)

In [ ]:
l1_train_re, l1_test_r2 = validation_curve(lasso, X_train, y_train, param_name="alpha", cv=age_skf, 
                                           param_range=alpha_range, scoring='r2', n_jobs=-1)
l1_train_mse, l1_test_mse = validation_curve(lasso, X_train, y_train, param_name="alpha", 
                                             cv=age_skf, param_range=alpha_range,  
                                             scoring='neg_mean_squared_error', n_jobs=-1)
# Save results in dataframe for display
valcurve_df = pd.concat([valcurve_to_df(l1_train_re, l1_test_r2, alpha_range, 'lasso', 'r2'),
                         valcurve_to_df(l1_train_mse, l1_test_mse, alpha_range, 'lasso', 'mse')])

In [ ]:
for mod_name in valcurve_df['model'].unique():
    kf_df = valcurve_df.loc[valcurve_df['model'] == mod_name]
    with sns.plotting_context("notebook", font_scale=1.2):
        g = sns.catplot(x='param', y='score', hue='stage', row='scorer', data=kf_df, kind='point',
                        sharey=False, hue_order =["train", "test"], height=3, aspect=3.0)
        #plt.xticks(range(10))
        g.set_xticklabels(alpha_range, rotation=90)
        #g.axes[0,0].set_ylim(-0.2, 1.1)
        #g.axes[0,1].set_ylim(-45, 1)
        g.fig.suptitle(mod_name, y=1.05)

In [ ]:
lasso_choice = Lasso(alpha=0.045, random_state=42, max_iter=100000)
y_pred = cross_val_predict(lasso_choice, X_train, y_train, cv=age_skf)

In [ ]:
# Scores
r2 = r2_score((y_train), (y_pred))
mse = mean_squared_error((y_train), (y_pred))
print("Scores from CV are R2: {}, MSE: {}".format(r2, mse))

sns.regplot((y_pred), (y_train))
plt.xlabel('Predicted Age with lasso')
plt.ylabel('True age')

#### Better approach: nested cross-validation

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
LassoCV?

In [ ]:
lasso_cv = LassoCV(alphas=alpha_range, random_state=42, max_iter=10000, cv=10, n_jobs=-1)

In [ ]:
%%capture output --no-stdout --no-display
nestedcv_results = []
best_alphas = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for i_f, (ix_train, ix_test) in enumerate(skf.split(X_train, age_class_train)):
        lasso_cv.fit(X_train[ix_train], y_train.iloc[ix_train])
        best_alphas.append(lasso_cv.alpha_)
        # Test on both the testing fold and training folds to check for over-/under-fitting
        y_pred_train = lasso_cv.predict(X_train[ix_train])
        y_pred_test = lasso_cv.predict(X_train[ix_test])
        # R2
        nestedcv_results.append({'model': 'lasso', 'fold': i_f, 'stage': 'train', 'scorer': 'r2', 
                                 'val': r2_score(y_train.iloc[ix_train], y_pred_train)})
        nestedcv_results.append({'model': 'lasso', 'fold': i_f, 'stage': 'test', 'scorer': 'r2', 
                                 'val': r2_score(y_train.iloc[ix_test], y_pred_test)})
        # MSE
        nestedcv_results.append({'model': 'lasso', 'fold': i_f, 'stage': 'train', 'scorer': 'MSE', 
                                 'val': -mean_squared_error(y_train.iloc[ix_train], y_pred_train)})
        nestedcv_results.append({'model': 'lasso', 'fold': i_f, 'stage': 'test', 'scorer': 'MSE', 
                                 'val': -mean_squared_error(y_train.iloc[ix_test], y_pred_test)})
nestedcv_results_df = pd.DataFrame(nestedcv_results)

In [ ]:
mod_name = 'lasso'
skf_df = nestedcv_results_df.loc[nestedcv_results_df['model'] == mod_name]
with sns.plotting_context("notebook", font_scale=1.2):
    # Barploat
    g = sns.catplot(x="fold", y="val", hue="stage", col="scorer", data=skf_df, kind="bar", 
                    sharey=False, height=3, aspect=1.5)
    g.axes[0,0].set_ylim(-0.2,1)
    g.fig.suptitle(mod_name, y=1.05)
    # Boxplot
    g = sns.catplot(x="model", y="val", hue="stage", col="scorer", data=skf_results_df, kind="box",
                    sharey=False, height=3, aspect=1.5)
    g.axes[0, 0].set_ylim(-0.2, 1)

In [ ]:
best_alphas

## Part 2: analyzing our final model

### Fit on whole (training) dataset 

In [ ]:
lasso_choice = Lasso(alpha=0.045, random_state=42, max_iter=100000)
lasso_choice.fit(X_train, y_train)

### Test and score on hold-out test dataset

In [ ]:
y_pred = lasso_choice.predict(X_test)

In [ ]:
# Scores
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print("Scores from CV are R2: {}, MSE: {}".format(r2, mse))

sns.regplot(y_pred, y_test)
plt.xlabel('Predicted Age with final lasso')
plt.ylabel('True age')

### Interpret model

In [ ]:
lasso_choice.coef_.shape

In [ ]:
plt.figure(figsize=(12, 6))
plt.stem(np.arange(len(lasso_choice.coef_)), lasso_choice.coef_)
plt.xlabel('Feature')
plt.ylabel('Beta coefficient')
plt.title("Final lasso model with {} parameters".format(np.count_nonzero(lasso_choice.coef_)));

### Want to see a brain plot of the connections prediction age? Let's dive into Nilearn!

# Nilearn usage example for resting-state fMRI connectivity

For this section we will follow closely the notebook created by Jake Vogel and presented at the 2020 Brainhack School: https://github.com/neurodatascience/course-materials-2020/blob/master/lectures/14-may/03-intro-to-machine-learning/ML_Regression_Tutorial.ipynb

Given $N$ regions of the brain, a functional connectivity matrix is typically created by:
* choosing a partition of the brain into $N$ regions (often non-overlapping)
* computing the correlation in time between each possible pair of regions $(i, j)$
* created the associated matrix where each element $r(i(t),j(t))$ of the matrix is the correlation $r$ between:
    * the signal $i(t)$ in region $i$
    * the signal $j(t)$ in region $j$

Standard pearson correlation is often used, so since $r(i(t), j(t)) = r(j(t), i(t))$ the matrix is symmetric, and since $r(k(t), k(t)) = 1$, the diagonal is made of $1$.

The data required to generate the connectivity matrices are fMRI data already preprocessed for motion, etc. The data is typically 4D: a 3D brain volume within which each voxel (3D pixel) includes a time course of that voxel activity. The activity is typically "rest": subjects just have to lie in the scanner during the scan.

With `nilearn` we can use the preprocessed data and transform them into data ready to be used with machine learning (the feature matrix `X` and label vector `y` we used before) with the following steps:
* Choose an atlas of $N$ regions (already aligned with all subjects)
* For each subject:
    * Extract the mean (or median) signal within each atlas region
    * Compute the correlation between each pair of regions
    * Transform the matrix into a vector: the features for that subject
* Concatenate all vectors as rows of the feature matrix `X`
* Create the label vector `y`, one value per subject:
    * a continuous value for regression (as we did before)
    * *or* a categorical value for classification (cf e.g. of classification in section 5)

We will use the 64-region version of the multi-resolution MIST atlas (cf. https://mniopenresearch.org/articles/1-3).

## Get and view the data

In [ ]:
from nilearn import datasets

In [ ]:
from nilearn import plotting

### Atlas

In [ ]:
atlases = datasets.fetch_atlas_basc_multiscale_2015(version='sym', 
                                                    data_dir='datasets/age_netmats')
atlas64_path = atlases.scale064
# Get the 3D coordinates of the atlas regions for future plotting
atlas64_coords = plotting.find_parcellation_cut_coords(atlas64_path)

#### Visualization

In [ ]:
plotting.plot_roi(atlas64_path, draw_cross=False);

In [ ]:
plotting.view_img(atlas64_path, cmap=plotting.cm.bwr, symmetric_cmap=False)

### MRI data

Let's have a look at the first subject

In [ ]:
### Already done earlier:
# datasets.fetch_development_fmri?
# data = datasets.fetch_development_fmri(data_dir='datasets/age_netmats')
s0_fmri_files = data.func[0]

#### Visualization

In [ ]:
from nilearn import image 

In [ ]:
s0_avg_vol = image.mean_img(s0_fmri_files)
display = plotting.plot_epi(s0_avg_vol, cmap="gray", vmin=0, vmax=1000)
#display.close()

In [ ]:
plotting.view_img?

In [ ]:
%%capture output --no-stdout --no-display
plotting.view_img(s0_avg_vol, cmap="binary", symmetric_cmap=False, threshold=0, vmin=0, vmax=1000)

## Create data in machine learning format

### Extract the time series from each brain region

In [ ]:
from nilearn.input_data import NiftiLabelsMasker

A `NiftiLabelsMasker` object can be used not only to extract the signal from each region but also to "clean it" by regressing out "confounds".  
In this case a confound file needs to be passed to `NiftiLabelsMasker `.

In [ ]:
s0_confounds_file = data.confounds[0]
s0_confounds_df = pd.read_csv(s0_confounds_file, sep='\t')
s0_confounds_df

In [ ]:
masker = NiftiLabelsMasker(labels_img=atlas64_path, standardize=True, 
                           memory='nilearn_cache',  verbose=1)
s0_time_series = masker.fit_transform(s0_fmri_files, confounds=s0_confounds_file)

Can you guess the size of the resulting `time_series` 2D array knowing that one dimension is the number of activity time points (168)?  
(Hint: try to remember the number of regions in the atlas)

In [ ]:
s0_time_series.shape

We can visualize the mean signal over some ROIS for subject 0

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(s0_time_series[:, 0], alpha=0.7, label='region 0')
plt.plot(s0_time_series[:, 1], alpha=0.7, label='region 1')
plt.plot(s0_time_series[:, 2], alpha=0.7, label='region 2')
plt.title('Voxel Time Series')
plt.xlabel('Time points (one 3D volume per time point)')
plt.ylabel('Normalized signal')
plt.legend()

Are the time series correlated? Let's quantify this

In [ ]:
np.corrcoef(s0_time_series[:, :3].T)

### Compute correlation matrix

In [ ]:
from nilearn.connectome import ConnectivityMeasure

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')
s0_conmat = correlation_measure.fit_transform([s0_time_series])[0]
s0_conmat.shape

In [ ]:
plotting.plot_matrix(s0_conmat, figure=(10, 8), labels=range(s0_time_series.shape[-1]),
                     vmax=0.8, vmin=-0.8, reorder=False)

In [ ]:
plotting.plot_connectome(s0_conmat, atlas64_coords, title='Functional connectivity of subject 0')

The next step is to process all the subjects. We will not do it as we already analyzed these data. The code is left below for reference.  
Note: the option `vectorize` is set to `True` in order to have the whole correlation matrix as a single vector of features (for each subject).

In [ ]:
masker = NiftiLabelsMasker(labels_img=atlas64_path, standardize=True, 
                           memory='nilearn_cache', verbose=0)
correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True, discard_diagonal=True)
vectorized_conmats = []
#for i_s, s in enumerate(data.func):
#    print("=== Processing subject {:03d}".format(i_s))
#    # Get the mean time series within each brain region
#    s_time_series = masker.fit_transform(s, confounds=data.confounds[i_s])
#    # Compute the correlation between each pair of region to create the connectivity matrix
#    s_conmat = correlation_measure.fit_transform([s_time_series])[0]
#    # Add to the list of vectorized connectivity matrices
#    vectorized_conmats.append(s_conmat)

## Visualize our results from the previous section

We'll first transform the vectorized `lasso` coefficients back to the matrix format (this is the inverse transform of what we just did).

In [ ]:
# Refit correlation to make sure the mat --> vec transform is calculated
correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True, discard_diagonal=True)
s0_conmat_vec = correlation_measure.fit_transform([s0_time_series])[0]
# Now we can apply the inverse transform
lasso_coeff_mat = correlation_measure.inverse_transform([lasso_choice.coef_])[0]

In [ ]:
plotting.plot_matrix(lasso_coeff_mat, figure=(10, 8), labels=range(lasso_coeff_mat.shape[0]),
                     reorder=False)

In [ ]:
plotting.plot_connectome(lasso_coeff_mat, atlas64_coords, colorbar=True)